In [3]:
import os, sys
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import numpy as np
import torch
import h5py
from scipy import signal

sys.path.append('/home/vanveen/ConvDecoder/')
from utils.data_io import load_h5_qdess

In [7]:
def get_file_list(path):
    return [f for f in listdir(path) if isfile(join(path, f))]

def blurring(F):
    """This blurs the image in three directions

    Parameters
    ----------
    F: array_like
        3D image to be blurred

    Returns
    -------
    images: tuple (array_like, array_like, array_like)
        This is a tuple of the three blurred images
    """
    
    length = 9
    hv = np.ones((length,1,1))/float(length)
    hh = np.ones((1,length,1))/float(length)
    hd = np.ones((1,1,length))/float(length)

    B_v = signal.convolve(F,hv,mode="same")
    B_h = signal.convolve(F,hh,mode="same")
    B_d = signal.convolve(F,hd,mode="same")
    
    return B_v, B_h, B_d

def blur_estimate(F):
    """Computes the blurring for some 3D image

    Parameters
    ----------
    F: array_like
        3D image to be blurred

    Returns
    -------
    blur_factor: float
        The maximum blurring
    """
    
    B_v, B_h, B_d = blurring(F)
    
    D_Fv = abs(F - np.roll(F,1,axis=0))
    D_Fh = abs(F - np.roll(F,1,axis=1))
    D_Fd = abs(F - np.roll(F,1,axis=2))
    
    D_Bv = abs(B_v - np.roll(B_v,1,axis=0))
    D_Bh = abs(B_h - np.roll(B_h,1,axis=1))
    D_Bd = abs(B_d - np.roll(B_d,1,axis=2))

    Vv = np.maximum(0,D_Fv-D_Bv)
    Vh = np.maximum(0,D_Fh-D_Bh)
    Vd = np.maximum(0,D_Fd-D_Bd)

    s_Fv = np.sum(D_Fv)
    s_Fh = np.sum(D_Fh)
    s_Fd = np.sum(D_Fd)
    
    s_Vv = np.sum(Vv)
    s_Vh = np.sum(Vh)
    s_Vd = np.sum(Vd)

    b_Fv = (s_Fv - s_Vv)/s_Fv
    b_Fh = (s_Fh - s_Vh)/s_Fh
    b_Fd = (s_Fd - s_Vd)/s_Fd

    blur_array = np.array([b_Fv, b_Fh, b_Fd])

    print("blurring in direction y: %.4f, x: %.4f, z: %.4f" %
          (b_Fv, b_Fh, b_Fd))

    
    blur= np.amax(blur_array)
    index = np.argmax(blur_array)
    
    return blur, index

# TODO

- i've reconstructed a 2d slice. how to use this?
    - over a volume of adjacent slices? how many?
    - modify it to work on 2d?
- look at paper

In [23]:
id_ = '005'
e_num = 2
accel = 4

path = '/bmrNAS/people/dvv/out_qdess/accel_{}x/echo_joint/specific_slices/'.format(accel)
files_all = get_file_list(path)

files = [f for f in files_all if '_e{}_'.format(e_num) in f and '_{}_'.format(id_) in f]
files.sort()

# im_dc = np.load('{}MTR_{}_e{}_dc.npy'.format(path_bl, id_, e_num)) 

In [25]:
arr_3d = np.empty((len(files), 512, 160))

for idx, f in enumerate(files):
    arr_3d[idx] = np.load(path + f)
    
print(arr_3d.shape)

(20, 512, 160)


In [26]:
blur_estimate(arr_3d)

blurring in direction y: 0.2206, x: 0.1516, z: 0.2140


(0.2206051052694979, 0)

In [16]:
blur_estimate(arr_3d)

blurring in direction y: 0.2099, x: 0.2344, z: 0.2639


(0.2638992309406652, 2)